Ways to get second highest salary

1.Using max and limit

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("SecondHighestSalary").getOrCreate()

# Sample data
data = [(1, 5000), (2, 6000), (3, 8000), (4, 8000), (5, 7000)]
columns = ["id", "salary"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("employees")

# SQL query to find second-highest salary
query = """
SELECT Max (salary)
FROM employees where salary<(SELECT Max (salary)
FROM employees)
LIMIT 1
"""

# Execute SQL query
result = spark.sql(query)
result.show()


using dense Rank

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("SecondHighestSalary").getOrCreate()

# Sample data
data = [(1, 5000), (2, 6000), (3, 8000), (4, 8000), (5, 7000)]
columns = ["id", "salary"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("employees")

# SQL query to find second-highest salary
query = """
SELECT salary 
FROM (
    SELECT salary, DENSE_RANK() OVER (ORDER BY salary DESC) as rnk
    FROM employees
) ranked
WHERE rnk = 2
"""

# Execute SQL query
result = spark.sql(query)
result.show()


Using Rank ****Key Differences Between RANK() and DENSE_RANK()
Function	Handles Duplicates?	Rank Gaps?
RANK()	Yes	Skips ranks when duplicates exist
DENSE_RANK()	Yes	No rank gaps
For example, if salaries are (8000, 8000, 7000, 6000, 5000):

RANK() results: (8000 → rank 1, 8000 → rank 1, 7000 → rank 3, 6000 → rank 4)
DENSE_RANK() results: (8000 → rank 1, 8000 → rank 1, 7000 → rank 2, 6000 → rank 3)
So, if you want the second-highest salary even when there are duplicates, DENSE_RANK() is safer

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("SecondHighestSalary").getOrCreate()

# Sample data
data = [(1, 5000), (2, 6000), (3, 8000), (4, 8000), (5, 7000)]
columns = ["id", "salary"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("employees")

# SQL query using RANK() with DISTINCT
query = """
SELECT salary 
FROM (
    SELECT DISTINCT salary, RANK() OVER (ORDER BY salary DESC) as rnk
    FROM employees
) ranked
WHERE rnk = 2
"""

# Execute SQL query
result = spark.sql(query)
result.show()


Using LAG() Window Function

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SecondHighestSalary").getOrCreate()

# Sample data
data = [(1, 5000), (2, 6000), (3, 8000), (4, 8000), (5, 7000)]
columns = ["id", "salary"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("employees")

query = """
SELECT max(salary) 
FROM (
    SELECT salary, LAG(salary) OVER (ORDER BY salary DESC) AS prev_salary
    FROM (SELECT DISTINCT salary FROM employees)
) t
WHERE prev_salary IS NOT NULL
"""

result = spark.sql(query)
result.show()


Using stored procedure

In [0]:
# Create sample data
data = [(1, 5000), (2, 6000), (3, 8000), (4, 8000), (5, 7000)]
columns = ["id", "salary"]

df = spark.createDataFrame(data, columns)

# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("employees")

# Create a stored procedure alternative (function)
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW second_highest_salary AS 
SELECT salary FROM (
    SELECT DISTINCT salary, DENSE_RANK() OVER (ORDER BY salary DESC) as rnk
    FROM employees
) ranked WHERE rnk = 2
""")
result = spark.sql("SELECT * FROM second_highest_salary")
result.show()


+------+
|salary|
+------+
|  7000|
+------+

